# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

In [2]:
! pip install kaggle

import os
import zipfile
os.environ['KAGGLE_USERNAME'] = "chaamp"
os.environ['KAGGLE_KEY'] = "26be8486f6a035048aa3e8c49d9001cd"


! kaggle datasets download -d chaamp/language-recognition

path='language-recognition.zip'
tar='Users/odl_user_141832/'
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('')

language-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
! pip install openpyxl

# Feature Extraction

In [15]:
# Feature extraction

df=pd.read_excel('trn.xlsx',engine='openpyxl',index_col=0)


In [17]:
df.head()

,0,1
0,I'll forgive you.,0
1,You're wise.,0
2,It's hideous.,0
3,I'm not well.,0
4,You seem upset.,0


In [27]:
from collections import Counter
import re
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


def three_gram(a):
  ans=[]
  for i in a:
    tmp=[]
    for j in i:
      n=len(j)
      if n>2:
        for k in range(n-2):
          tmp.append(j[k:k+3])
    ans.append(tmp)
  return ans

def create_vocab(data):
  df=data.copy()
  a=df.groupby(1)
  d={}
  for i,g in a:
    d[i]=g[0].tolist()

  for i in range(8):
    # Lower case the strings
    d[i]=[x.lower() for x in d[i]]
    
    # Remove spacial character and numbers
    d[i]=[re.sub('\W+',' ', x) for x in d[i]]
    d[i]=[re.sub(r'\d+', '', x) for x in d[i]]
    
    # strip the strings
    d[i]=list(map(str.strip,d[i]))

    #split the strings
    d[i]=[x.split() for x in d[i]]

  # Create a vocabulary of 1st 250 character 3-gram 

  # create a counter
  count_vec={}
  for i in range(8):
    x=three_gram(d[i])
    c=Counter()
    x=[Counter(k) for k in x]
    for j in x:
      c=c+j
    count_vec[i]=c
  # sort the keys and vocab
  voc={}
  vocab=[]
  for i in range(8):
    count_vec[i]={k: v for k, v in sorted(count_vec[i].items(), key=lambda item: item[1],reverse=True)}
    l=list(count_vec[i].keys())[:200]
    vocab=vocab+l
    voc[i]=l
  vocab=list(set(vocab))
  return vocab


def feature_extraction(data):
  #group the data by languages
  trn=data.copy()
  a=trn.groupby(1)
  ftr={}
  vocab=create_vocab(trn)
  vectorizer=CountVectorizer(analyzer='char',ngram_range=(3,3),vocabulary=vocab)
  for i,g in a:
    x=list(set(g[0].tolist()))
    ftr[i]=vectorizer.fit_transform(x).toarray()
  for i in range(8):
    ftr[i]=ftr[i][~np.all(ftr[i]==0,axis=1)]

# Combine all the arrays

  # create labels
  lbl={}
  ftr_={}
  for i in range(8):
    lbl[i]=np.array([i]*ftr[i].shape[0]).reshape((ftr[i].shape[0],1))
    ftr_[i]=np.append(ftr[i],lbl[i],1)
    if i==0:
      train=ftr_[i]
    else:
      train=np.append(train,ftr_[i],0)

  #Shuffle the dataset
  np.random.shuffle(train)

  trainx=train[:,:-1]
  trainy=train[:,-1]

  return train

In [28]:
x=feature_extraction(df)

In [ ]:
x.shape

In [4]:
# Import the data
from azureml.core.dataset import Dataset
import pandas as pd


key = 'LR data'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found")
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])

else:
    df=pd.read_excel('trn.xlsx',engine='openpyxl',index_col=0)
    df.to_csv('trn.csv',index = False)
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['./trn.csv'])
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])
    train = train.register(ws,key)

Dataset found


# Define Compute instance

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy",
                   "experiment_timeout_minutes":20,
                   "enable_early_stopping":True,
                   'enable_tf':True,
                   'enable_dnn':True,
                   'blocked_models': None,
                   "max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name ='1',**automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_4a93314b-61c5-471f-ba41-6a4622ee0689

Current status: FeaturesGeneration. Generating features for the dataset.
Something went wrong while printing the experiment progress but the run is still executing on the compute target. 
Please check portal for updated status: https://ml.azure.com/experiments/CapstoneProject/runs/AutoML_4a93314b-61c5-471f-ba41-6a4622ee0689?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-141913/workspaces/quick-starts-ws-141913


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service